In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import xgboost as xgb
import tensorflow as tf
from gc import collect
import pandas as pd
import os
checkpoint_path = "training_ensemble/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


In [2]:
df_data=pd.read_csv("DataSetNormalised.csv")
df_label=pd.read_csv("LabelSet.csv")

In [5]:
x=np.array(df_data.iloc[:,1:])
y=np.array(df_label.iloc[:,1])
x=x.astype(np.float32)
x_test=x[10000000:11000000][:]
y_test=y[10000000:11000000][:]
x_train=x[:1000000][:]
y_train=y[:1000000][:]

In [6]:
del df_data
del df_label
del x
del y
collect()

87

In [7]:
model_xgb=XGBClassifier()
model_xgb.load_model("XGBoost.json")
y_test_pred_xgb=model_xgb.predict_proba(x_test)
y_train_pred_xgb=model_xgb.predict_proba(x_train)
y_pred_xgb=y_test_pred_xgb[:,1]
y_pred_xgb_train=y_train_pred_xgb[:,1]

In [8]:
y_test_pred_xgb=np.reshape(y_pred_xgb,(1000000,1))
y_train_pred_xgb=np.reshape(y_pred_xgb_train,(1000000,1))
y_train_pred_xgb.shape

(1000000, 1)

In [9]:
model_DNN_5 = tf.keras.models.load_model('DNN_5')
y_test_pred_DNN_5=model_DNN_5.predict(x_test)
y_train_pred_DNN_5=model_DNN_5(x_train)

In [10]:
model_NN_Shallow_1024 = tf.keras.models.load_model('NN_Shallow_1024')
y_test_pred_NN_Shallow_1024=model_NN_Shallow_1024.predict(x_test)
y_train_pred_NN_Shallow_1024=model_NN_Shallow_1024.predict(x_train)

In [11]:
model_NN_Shallow_2048 = tf.keras.models.load_model('NN_Shallow_2048')
y_test_pred_NN_Shallow_2048=model_NN_Shallow_2048.predict(x_test)
y_train_pred_NN_Shallow_2048=model_NN_Shallow_2048.predict(x_train)

In [12]:
model_NN_Shallow_10240 = tf.keras.models.load_model('NN_Shallow_10240')
y_test_pred_NN_Shallow_10240=model_NN_Shallow_10240.predict(x_test)
y_train_pred_NN_Shallow_10240=model_NN_Shallow_10240.predict(x_train)

In [13]:
model_ResNet_52 = tf.keras.models.load_model('ResNet_52')
y_test_pred_model_ResNet_52=model_ResNet_52.predict(x_test)
y_train_pred_model_ResNet_52=model_ResNet_52.predict(x_train)

In [14]:
df_entrain=pd.DataFrame(x_train)
df_entrain['XGB']=y_train_pred_xgb
df_entrain['DNN5']=y_train_pred_DNN_5
df_entrain['S1024']=y_train_pred_NN_Shallow_1024
df_entrain['S2048']=y_train_pred_NN_Shallow_2048
df_entrain['10240']=y_train_pred_NN_Shallow_10240
df_entrain['RN52']=y_train_pred_model_ResNet_52
df_entrain.to_csv("Train_ens.csv")

In [15]:
df_entest=pd.DataFrame(x_test)
df_entest['XGB']=y_test_pred_xgb
df_entest['DNN5']=y_test_pred_DNN_5
df_entest['S1024']=y_test_pred_NN_Shallow_1024
df_entest['S2048']=y_test_pred_NN_Shallow_2048
df_entest['10240']=y_test_pred_NN_Shallow_10240
df_entest['RN52']=y_test_pred_model_ResNet_52
df_entest.to_csv("Test_ens.csv")

In [16]:
x_test_en=np.array(df_entest.iloc[:,28:])

x_train_en=np.array(df_entrain.iloc[:,28:])
x_test_en.shape

(1000000, 6)

In [156]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)


In [152]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(512,activation='gelu'),
    tf.keras.layers.Dense(512,activation='gelu'),
    tf.keras.layers.Dense(128,activation='gelu'),
    tf.keras.layers.Dense(128,activation='gelu'),
    tf.keras.layers.Dense(10,activation='gelu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

In [155]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.0001,
        beta_1=0.9,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

In [157]:


cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(
    x_train_en,
    y_train,
    epochs=100,
    batch_size=10240,
    validation_data=(x_test_en,y_test),
    callbacks=[cp_callback]
)

Epoch 1/100
98/98 [==============================] - ETA: 0s - loss: 0.3533 - accuracy: 0.8427 - auc: 0.9222
Epoch 00001: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 3s 20ms/step - loss: 0.3533 - accuracy: 0.8427 - auc: 0.9222 - val_loss: 0.4812 - val_accuracy: 0.7745 - val_auc: 0.8608
Epoch 2/100
96/98 [============================>.] - ETA: 0s - loss: 0.3530 - accuracy: 0.8428 - auc: 0.9223
Epoch 00002: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 2s 18ms/step - loss: 0.3529 - accuracy: 0.8428 - auc: 0.9224 - val_loss: 0.4817 - val_accuracy: 0.7756 - val_auc: 0.8604
Epoch 3/100
96/98 [============================>.] - ETA: 0s - loss: 0.3531 - accuracy: 0.8428 - auc: 0.9223
Epoch 00003: saving model to training_ensemble\cp.ckpt
98/98 [==============================] - 2s 18ms/step - loss: 0.3530 - accuracy: 0.8428 - auc: 0.9223 - val_loss: 0.4841 - val_accuracy: 0.7729 - val_auc: 0.8589
Epoch 4/100
96/98 [===

KeyboardInterrupt: 

In [150]:
pre_max=np.random.rand(6,1)
auc_max=0

ValueError: operands could not be broadcast together with shapes (6,1000000,1) (6,1) 

In [182]:
from time import time

for i in range(10000):
    pre=(np.random.rand(6,1))
    y_pred_ensemble=y_test_pred_DNN_5*pre[0]
    y_pred_ensemble=y_pred_ensemble+y_test_pred_model_ResNet_52*pre[1]
    y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_1024*pre[2]
    y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_10240*pre[3]
    y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_2048*pre[4]
    y_pred_ensemble=y_pred_ensemble+y_test_pred_xgb*pre[5]
    y_pred_ensemble=y_pred_ensemble/(pre[0]+pre[1]+pre[2]+pre[3]+pre[4]+pre[5])
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_ensemble)
    auc=metrics.auc(fpr, tpr)
    if auc>auc_max:
        auc_max=auc
        print("New Max detected")
        print("Auc",auc)
        print("--------------------------------------------")
        pre_max=pre
    if i%1000==0:
        print(str(i))
print(str(t2-t1))

#print("The training accuracy",str(accuracy_score(y_test`, np.round(y_pred_ensemble))))

0
1000
New Max detected
Auc 0.8798746822044039
--------------------------------------------
New Max detected
Auc 0.8800095770975933
--------------------------------------------
2000
3000
New Max detected
Auc 0.8800408709888439
--------------------------------------------
4000
5000
New Max detected
Auc 0.8802592341922955
--------------------------------------------
6000
7000


KeyboardInterrupt: 

In [184]:
pre_max

array([[0.98742091],
       [0.51091043],
       [0.40462122],
       [0.00178943],
       [0.18467257],
       [0.17939743]])

In [ ]:
#Trying out average ensemble
y_pred_ensemble=y_test_pred_DNN_5+0*y_test_pred_NN_Shallow_1024
y_pred_ensemble=np.round(y_pred_ensemble)

accuracy = accuracy_score(y_test, y_pred_ensemble)
print("The training accuracy",str(accuracy))
fpr, tpr, thresholds = roc_curve(y_test, y_pred_ensemble)
metrics.auc(fpr, tpr)
